In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('data/cleaned_reviews.csv')
df.head()

,Movie_N,cleaned_review,Stars
0,The Old Guard,love concept execution not bad there some crin...,7
1,The Old Guard,there were just so many clichés cringe naive m...,5
2,The Old Guard,actors all good well thats story childlike dir...,5
3,The Old Guard,my mind kept referring back austin powers badd...,9
4,The Old Guard,london scenes end were just bizarre empty stre...,5


In [4]:
def get_sentiment_label(stars):
    stars=int(stars)
    if stars <= 4:
        return 0  # Negative
    elif stars <= 6:
        return 1  # Neutral
    else:
        return 2  # Positive

In [5]:
df.dropna(subset=['cleaned_review'], inplace=True)


In [6]:
df['Stars']=df['Stars'].astype(int)

In [7]:
df['label'] = df['Stars'].apply(get_sentiment_label)
df.to_csv("data/labeled_reviews.csv", index=False)
print("✅ Labeled reviews with 3 sentiment classes saved.")

✅ Labeled reviews with 3 sentiment classes saved.


In [8]:
df

,Movie_N,cleaned_review,Stars,label
0,The Old Guard,love concept execution not bad there some crin...,7,2
1,The Old Guard,there were just so many clichés cringe naive m...,5,1
2,The Old Guard,actors all good well thats story childlike dir...,5,1
3,The Old Guard,my mind kept referring back austin powers badd...,9,2
4,The Old Guard,london scenes end were just bizarre empty stre...,5,1
...,...,...,...,...
5676,Greyhound,my only complaint movie could have been longer...,5,1
5677,Greyhound,though movie might have been right technical t...,8,2
5678,Greyhound,i felt movie could have been longer better dep...,1,0
5679,Greyhound,climax if any not up mark,9,2


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import joblib
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier  # BEGIN:

from sklearn.svm import LinearSVC  # END:

In [10]:
X_text = df["cleaned_review"]
y = df["label"]

In [11]:
vector=joblib.load("models/tfidf_vectorizer.pkl")
X=vector.fit_transform(X_text)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42)

In [29]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [500, 1000]
}
grid = GridSearchCV(LogisticRegression(multi_class="multinomial"), param_grid, cv=3, scoring="accuracy")
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
model = grid.best_estimator_


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, 

Best parameters: {'C': 0.01, 'max_iter': 500, 'solver': 'lbfgs'}


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [14]:
models = {
    "Logistic Regression": LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "Naive Bayes": MultinomialNB(),
    "Linear SVM": LinearSVC()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        "accuracy": report["accuracy"],
        "macro avg f1-score": report["macro avg"]["f1-score"]
    }
results_df = pd.DataFrame(results).T
print(results_df.sort_values(by="macro avg f1-score", ascending=False))


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: Unde

                     accuracy  macro avg f1-score
XGBoost              0.611650            0.324499
Logistic Regression  0.637246            0.318562
Linear SVM           0.636364            0.312677
Random Forest        0.638129            0.309144
Naive Bayes          0.632833            0.258378


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: Unde

In [30]:

model.score(X_test, y_test)

0.6348314606741573

In [31]:
joblib.dump(model, "models/sentiment_model.pkl")
print("Model saved as sentiment_model.pkl")

Model saved as sentiment_model.pkl
